<a href="https://colab.research.google.com/github/wlg100/numseqcont_circuit_expms/blob/main/nb_templates/circuit_expms_template.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

This will save files to your local machine if `save_files` is set to True.

# Change Inputs Here

In [ ]:
task = "months"  # choose: numerals, numwords, months
prompt_types = ['done', 'lost', 'names']
num_samps_per_ptype = 512 #768 512

model_name = "gpt2-small"

# Without MLP 11
heads_not_ablate = [(0, 1), (0, 5), (4, 4), (6, 1), (6, 6), (6, 10), (7, 6), (7, 9), (7, 10), (7, 11), (8, 8), (9, 1), (10, 7)]
mlps_not_ablate = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10]

# With MLP 11
# heads_not_ablate = [(0, 1), (4, 4), (4, 10), (6, 1), (6, 6), (6, 10), (7, 2), (7, 10), (7, 11), (8, 8), (9, 1)]
# mlps_not_ablate = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11] # incl 5 makes it 66.1155%

threshold = 0.8
save_files = True
load_graph_files = False
run_on_other_tasks = True

# Setup

In [ ]:
%%capture
%pip install git+https://github.com/neelnanda-io/TransformerLens.git

In [ ]:
import torch as t
from torch import Tensor
from jaxtyping import Float
from typing import List, Optional, Callable, Tuple
from functools import partial

import pickle
from google.colab import files

from graphviz import Digraph, Source
from IPython.display import display

In [ ]:
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, ActivationCache

t.set_grad_enabled(False)

## Load Model

In [ ]:
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

## Import functions from repo

In [ ]:
!git clone https://github.com/apartresearch/seqcont_circuits.git

In [ ]:
%cd /content/seqcont_circuits/src/iter_node_pruning

from dataset import Dataset
from metrics import *
from head_ablation_fns import *
from mlp_ablation_fns import *
from node_ablation_fns import *
from loop_node_ablation_fns import *

In [ ]:
%cd /content/seqcont_circuits/src/iter_edge_pruning

from edge_pruning_fns import *
from viz_circuits import *

# Load datasets

In [ ]:
prompts_list = []

for i in prompt_types:
    file_name = f'/content/seqcont_circuits/data/{task}/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:num_samps_per_ptype]

len(prompts_list)

In [ ]:
pos_dict = {}
for i in range(len(model.tokenizer.tokenize(prompts_list[0]['text']))):
    pos_dict['S'+str(i)] = i

In [ ]:
dataset_1 = Dataset(prompts_list, pos_dict, model.tokenizer)

In [ ]:
file_name = f'/content/seqcont_circuits/data/{task}/randDS_{task}.pkl'
with open(file_name, 'rb') as file:
    prompts_list_2 = pickle.load(file)

In [ ]:
dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer)

## Metrics

In [ ]:
model.reset_hooks(including_permanent=True)
logits_original = model(dataset_1.toks)
orig_score = get_logit_diff(logits_original, dataset_1)

# model.reset_hooks(including_permanent=True)
# logits_corrup = model(dataset_2.toks)
# corrup_score = get_logit_diff(abc_logits_original, dataset_2)

In [ ]:
def logit_diff_perc(
    logits: Float[Tensor, "batch seq d_vocab"],
    clean_logit_diff: float = orig_score,
    dataset_1: Dataset = dataset_1,
) -> float:
    patched_logit_diff = get_logit_diff(logits, dataset_1)
    return (patched_logit_diff / clean_logit_diff)

In [ ]:
import gc

del(logits_original)
# del(logits_corrup)
t.cuda.empty_cache()
gc.collect()

# Run Iter Edge Pruning

## get circuit

Get the node ablated circuit without edges ablated.

In [ ]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
abl_model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

new_logits = model(dataset_1.toks)
new_score = get_logit_diff(new_logits, dataset_1)
circ_score = (100 * new_score / orig_score).item()
print(f"(cand circuit / full) %: {circ_score:.4f}")
del(new_logits)

## head to head

In [ ]:
qkv_to_HH = {} # qkv to dict

for head_type in ["q", "k", "v"]:
    head_to_head_results = {}
    for head in heads_not_ablate:
        print(head_type, head)
        model.reset_hooks()
        model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

        result = circ_path_patch_head_to_heads(
            circuit = heads_not_ablate,
            receiver_heads = [head],
            receiver_input = head_type,
            model = model,
            patching_metric = logit_diff_perc,
            new_dataset = dataset_2,
            orig_dataset = dataset_1
        )
        head_to_head_results[head] = result
    qkv_to_HH[head_type] = head_to_head_results

In [ ]:
head_to_head_adjList = {}
for head_type in ["q", "k", "v"]:
    for head in heads_not_ablate:
        result = qkv_to_HH[head_type][head]
        filtered_indices = (result < threshold) & (result != 0.0)
        rows, cols = filtered_indices.nonzero(as_tuple=True)
        sender_nodes = list(zip(rows.tolist(), cols.tolist()))
        head_with_type = head + (head_type,)
        head_to_head_adjList[head_with_type] = sender_nodes

## mlp to mlp

In [ ]:
mlp_to_mlp_results = {}

for layer in reversed(mlps_not_ablate):
    print(layer)
    model.reset_hooks()
    model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)
    result = circ_path_patch_MLPs_to_MLPs(
        mlp_circuit = mlps_not_ablate,
        receiver_layers = [layer],
        model = model,
        patching_metric = logit_diff_perc,
        new_dataset = dataset_2,
        orig_dataset = dataset_1
    )
    mlp_to_mlp_results[layer] = result

In [ ]:
mlp_to_mlp_adjList = {}
for mlp in mlps_not_ablate:
    result = mlp_to_mlp_results[mlp]
    filtered_indices = (result < threshold) & (result != 0.0)
    filtered_indices = filtered_indices.nonzero(as_tuple=True)[0]
    mlp_to_mlp_adjList[mlp] = filtered_indices.tolist()

## head to mlp

In [ ]:
head_to_mlp_results = {}

for layer in reversed(mlps_not_ablate):
    print(layer)
    model.reset_hooks()
    model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)
    result = circ_path_patch_head_to_mlp(
        circuit = heads_not_ablate,
        receiver_layers = [layer],
        model = model,
        patching_metric = logit_diff_perc,
        new_dataset = dataset_2,
        orig_dataset = dataset_1
    )
    head_to_mlp_results[layer] = result

In [ ]:
head_to_mlp_adjList = {}
for layer in mlps_not_ablate:
    result = head_to_mlp_results[layer]
    filtered_indices = (result < threshold) & (result != 0.0)
    rows, cols = filtered_indices.nonzero(as_tuple=True)
    sender_nodes = list(zip(rows.tolist(), cols.tolist()))
    head_to_mlp_adjList[layer] = sender_nodes

## mlp to head

In [ ]:
qkv_mlp_to_HH = {} # qkv to dict

for head_type in ["q", "k", "v"]:
    mlp_to_head_results = {}
    for head in heads_not_ablate:
        print(head_type, head)
        model.reset_hooks()
        model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

        result = circ_path_patch_mlp_to_head(
            mlp_circuit = mlps_not_ablate,
            receiver_heads = [head],
            receiver_input = head_type,
            model = model,
            patching_metric = logit_diff_perc,
            new_dataset = dataset_2,
            orig_dataset = dataset_1
        )
        mlp_to_head_results[head] = result
    qkv_mlp_to_HH[head_type] = mlp_to_head_results

In [ ]:
mlp_to_head_adjList = {}
for head_type in ["q", "k", "v"]:
    for head in heads_not_ablate:
        result = qkv_mlp_to_HH[head_type][head]
        filtered_indices = (result < threshold) & (result != 0.0)
        filtered_indices = filtered_indices.nonzero(as_tuple=True)[0]
        head_with_type = head + (head_type,)
        mlp_to_head_adjList[head_with_type] = filtered_indices.tolist()

# Save graph files to free up memory

In [ ]:
if save_files:
    with open(task + "_head_to_head_results.pkl", "wb") as file:
        pickle.dump(head_to_head_results, file)
    files.download(task + "_head_to_head_results.pkl")

    with open(task + "_mlp_to_mlp_results.pkl", "wb") as file:
        pickle.dump(mlp_to_mlp_results, file)
    files.download(task + "_mlp_to_mlp_results.pkl")

    with open(task + "_head_to_mlp_results.pkl", "wb") as file:
        pickle.dump(head_to_mlp_results, file)
    files.download(task + "_head_to_mlp_results.pkl")

    with open(task + "_mlp_to_head_results.pkl", "wb") as file:
        pickle.dump(mlp_to_head_results, file)
    files.download(task + "_mlp_to_head_results.pkl")

del(head_to_head_results)
del(mlp_to_mlp_results)
del(head_to_mlp_results)
del(mlp_to_head_results)

# Run Iter Edge Pruning- resid post

## head to resid

In [ ]:
model.reset_hooks()
model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

path_patch_head_to_final_resid_post = get_path_patch_head_to_final_resid_post(heads_not_ablate, model, logit_diff_perc,
                                                                new_dataset = dataset_2, orig_dataset = dataset_1)

In [ ]:
heads_to_resid = {}
result = path_patch_head_to_final_resid_post
filtered_indices = (result < threshold) & (result != 0.0)
rows, cols = filtered_indices.nonzero(as_tuple=True)
heads_to_resid['resid'] = list(zip(rows.tolist(), cols.tolist()))

## mlp to resid

In [ ]:
model.reset_hooks()
model = add_ablation_hook_MLP_head(model, dataset_2, heads_not_ablate, mlps_not_ablate)

path_patch_mlp_to_final_resid_post = get_path_patch_mlp_to_final_resid_post(mlps_not_ablate, model, logit_diff_perc,
                                                                new_dataset = dataset_2, orig_dataset = dataset_1)

In [ ]:
mlps_to_resid = {}
result = path_patch_mlp_to_final_resid_post
filtered_indices = (result < threshold) & (result != 0.0)
filtered_indices = filtered_indices.nonzero(as_tuple=True)[0]
mlps_to_resid['resid'] = filtered_indices.tolist()

# Filter out nodes with no ingoing edges

In [ ]:
head_to_head_adjList = {node: neighbors for node, neighbors in head_to_head_adjList.items() if neighbors}

In [ ]:
mlp_to_head_adjList = {node: neighbors for node, neighbors in mlp_to_head_adjList.items() if neighbors}

# Save rest of graph files

In [ ]:
if save_files:
    # graphs
    with open(task + "_head_to_head_adjList.pkl", "wb") as file:
        pickle.dump(head_to_head_adjList, file)
    files.download(task + "_head_to_head_adjList.pkl")

    with open(task + "_mlp_to_mlp_adjList.pkl", "wb") as file:
        pickle.dump(mlp_to_mlp_adjList, file)
    files.download(task + "_mlp_to_mlp_adjList.pkl")

    with open(task + "_head_to_mlp_adjList.pkl", "wb") as file:
        pickle.dump(head_to_mlp_adjList, file)
    files.download(task + "_head_to_mlp_adjList.pkl")

    with open(task + "_mlp_to_head_adjList.pkl", "wb") as file:
        pickle.dump(mlp_to_head_adjList, file)
    files.download(task + "_mlp_to_head_adjList.pkl")

    with open(task + "_heads_to_resid.pkl", "wb") as file:
        pickle.dump(heads_to_resid, file)
    files.download(task + "_heads_to_resid.pkl")

    with open(task + "_mlps_to_resid.pkl", "wb") as file:
        pickle.dump(mlps_to_resid, file)
    files.download(task + "_mlps_to_resid.pkl")

    # score results
    with open(task + "_heads_to_resid_results.pkl", "wb") as file:
        pickle.dump(path_patch_head_to_final_resid_post, file)
    files.download(task + "_heads_to_resid_results.pkl")

    with open(task + "_mlps_to_resid_results.pkl", "wb") as file:
        pickle.dump(path_patch_mlp_to_final_resid_post, file)
    files.download(task + "_mlps_to_resid_results.pkl")

# Circuit graph plot

## load data

In [ ]:
if load_graph_files:
    with open(task + "_head_to_head_adjList.pkl", "rb") as file:
        head_to_head_adjList = pickle.load(file)

    with open(task + "_mlp_to_mlp_adjList.pkl", "rb") as file:
        mlp_to_mlp_adjList = pickle.load(file)

    with open(task + "_head_to_mlp_adjList.pkl", "rb") as file:
        head_to_mlp_adjList = pickle.load(file)

    with open(task + "_mlp_to_head_adjList.pkl", "rb") as file:
        mlp_to_head_adjList = pickle.load(file)

    with open(task + "_heads_to_resid.pkl", "rb") as file:
        heads_to_resid = pickle.load(file)

    with open(task + "_mlps_to_resid.pkl", "rb") as file:
        mlps_to_resid = pickle.load(file)

## plot qkv

In [ ]:
plot_graph_adjacency_qkv(head_to_head_adjList, mlp_to_mlp_adjList, head_to_mlp_adjList,
                         mlp_to_head_adjList, heads_to_resid, mlps_to_resid, filename="qkv")

## plot no qkv fn

In [ ]:
plot_graph_adjacency(head_to_head_adjList, mlp_to_mlp_adjList, head_to_mlp_adjList,
                         mlp_to_head_adjList, heads_to_resid, mlps_to_resid, filename="no qkv")